# Lending Club

### A complete data analysis project on loans issued by Lending Club

Lending Club is a website that allows investors to pool their money to issue loans to borrowers, with Lending Club acting as an intermediary on the transactions.  Interest rates are set by the borrowers credit score as well as the proprietary data analytics of Lending club.

Included in this folder is a data dictionary describing the data set.  The data dictionary is in XLS (Excel) format, and has a few tabs contained within.  The first tab is called "LoanStats" and only contains information on loans that were issued, ignoring all loans that were rejected.  This will include loans that defaulted, since they were issued in the first place, and current loans, in addition to loans that were satisfied by the borrower.

We will be attempting to build a machine learning model that can accurately predict whether a borrower will pay back their loan in a timely fashion. 

## Features and Labels

Before we build our model, we will need to identify any relevant columns of data that can be included in our feature-set for predictive power.  

We will also need to identify a "label" or what our model is *going to attempt to predict*.

We will only be using a calendar-range subset of the data from 2007-2011.  The reason for this is that loans made during that time period have mostly ended, so we will have a good idea how many of them have defaulted and how many have been satisfactorily paid. 

Let's take a look at the data.


In [81]:
# importing modules
import pandas as pd
import numpy as np

df = pd.read_csv('loanstats.csv', low_memory=False)

df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
# printing columns
for col in df.columns:
    print(col,": ", df[col].dtypes)


id :  object
member_id :  float64
loan_amnt :  float64
funded_amnt :  float64
funded_amnt_inv :  float64
term :  object
int_rate :  float64
installment :  float64
grade :  object
sub_grade :  object
emp_title :  object
emp_length :  object
home_ownership :  object
annual_inc :  float64
verification_status :  object
issue_d :  object
loan_status :  object
pymnt_plan :  object
url :  object
desc :  object
purpose :  object
title :  object
zip_code :  object
addr_state :  object
dti :  float64
delinq_2yrs :  float64
earliest_cr_line :  object
fico_range_low :  float64
fico_range_high :  float64
inq_last_6mths :  float64
mths_since_last_delinq :  float64
mths_since_last_record :  float64
open_acc :  float64
pub_rec :  float64
revol_bal :  float64
revol_util :  float64
total_acc :  float64
initial_list_status :  object
out_prncp :  float64
out_prncp_inv :  float64
total_pymnt :  float64
total_pymnt_inv :  float64
total_rec_prncp :  float64
total_rec_int :  float64
total_rec_late_fee :  floa

In [83]:
df.shape

(2260701, 151)

The data set is large, so we will eliminate all columns that are missing 50% or greater values.

We will remove the 'desc' column which contains a lengthy description of the reason the borrower requested the loan.

We will also remove the 'url' column which contains a link that investors backing the loan can use to check up on its progress.

In [84]:
# null values of each column
null_counts = df.isnull().sum()

# amount of rows in df
df_rows = len(df)

# list to hold removed columns
remove_cols = []

# loop to remove any columns with 50% or greater missing values
for col in df.columns:
    if null_counts[col] / df_rows >= .5:
        df.drop(col, axis=1, inplace=True)
        remove_cols.append(col)
        
        
# print all columns removed
remove_cols


['member_id',
 'desc',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'next_pymnt_d',
 'mths_since_last_major_derog',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'mths_since_recent_bc_dlq',
 'mths_since_recent_revol_delinq',
 'revol_bal_joint',
 'sec_app_fico_range_low',
 'sec_app_fico_range_high',
 'sec_app_earliest_cr_line',
 'sec_app_inq_last_6mths',
 'sec_app_mort_acc',
 'sec_app_open_acc',
 'sec_app_revol_util',
 'sec_app_open_act_il',
 'sec_app_num_rev_accts',
 'sec_app_chargeoff_within_12_mths',
 'sec_app_collections_12_mths_ex_med',
 'sec_app_mths_since_last_major_derog',
 'hardship_type',
 'hardship_reason',
 'hardship_status',
 'deferral_term',
 'hardship_amount',
 'hardship_start_date',
 'hardship_end_date',
 'payment_plan_start_date',
 'hardship_length',
 'hardship_dpd',
 'hardship_loan_status',
 'orig_projected_additional_accrued_interest',
 'hardship_payoff_balance_amount',
 'hardship_last_payment_amount',
 'debt_settlement_flag_date',
 'sett

In [ ]:
# importing datetime module
import datetime as dt

# converting dates to datetime objects
df['issue_d']= pd.to_datetime(df.issue_d)


In [ ]:
# removing all loans that start after 2011

df = df[df.issue_d <= dt.datetime(2011,12,31)]

In [ ]:
# checking to make sure our change took place

print('min: ',df.issue_d.min())
print('max: ',df.issue_d.max())

In [ ]:
# removing 'url' columns ( 'desc' was removed because it contained too many missing values)

df.drop('url', axis=1, inplace=True)

### Removing More Columns From Data

Because this data set contains so many columns, we are going to examine the first 18 columns and see if there are any that we can eliminate from our feature set, or choose as our label.

We can eliminate columns that:

- Leak data, or any information about a loan that was collected after the loan was given

- Have no effect on a borrowers ability to pay back the loan

- Require additional cleaning or processing to be useful as a feature

- Are duplicates or near-duplicates of other columns

### Here are the first 18 rows:

In [ ]:
first_18 = df.columns[0:18]
second_18 = df.columns[18:36]
third_18 = df.columns[36:54]

for col in first_18:
    print(col)

id: remove due to it being arbitrary

funded_amnt: removed because the loan has already been funded at this point

funded_amnt_inv: removed because the loan has already been funded at this point

grade: duplicate info of interest rate, with less granularity

sub_grade: duplicate info of interest rate, with less granularity

emp_title: would need to be categorized manually, which would take too much work on a large data set

issue_d: removed because the loan has already been funded at this point

In [ ]:
# removing columns

df.drop(['id',
         'funded_amnt',
         'funded_amnt_inv',
         'grade','sub_grade',
         'emp_title',
         'issue_d'], axis=1, inplace=True)


### Here are the next 18 rows:

In [ ]:
for col in second_18:
    print(col)

zip_code: only contains the first thee numbers, so dropped

out_prncp: removed because the loan has already been funded at this point 

out_prncp_inv: removed because the loan has already been funded at this point

total_pymnt: removed because the loan has already been funded at this point

total_pymnt_inv: removed because the loan has already been funded at this point

total_rec_prncp: removed because the loan has already been funded at this point


In [ ]:
# removing columns

df.drop(['zip_code',
         'out_prncp',
         'out_prncp_inv',
         'total_pymnt',
         'total_pymnt_inv',
         'total_rec_prncp'], axis=1, inplace=True)

In [ ]:
### Here are the next 18 rows:

for col in third_18:
    print(col)

total_rec_int: removed because the loan has already been funded at this point 

total_rec_late_fee: removed because the loan has already been funded at this point

recoveries: removed because the loan has already been funded at this point

collection_recovery_fee: removed because the loan has already been funded at this point

last_pymnt_d: removed because the loan has already been funded at this point

last_pymnt_amnt: removed because the loan has already been funded at this point

In [ ]:
# removing columns

df.drop(['total_rec_int',
         'total_rec_late_fee',
         'recoveries',
         'collection_recovery_fee',
         'last_pymnt_d',
         'last_pymnt_amnt'], axis=1, inplace=True)

In [ ]:
df.head()

Finally, from our remaining columns, we will choose a subset that look interesting for predictions...

In [ ]:
df = df[['loan_amnt', 'term', 'int_rate',
        'installment', 'emp_length', 'home_ownership',
        'annual_inc', 'verification_status', 'loan_status',
        'pymnt_plan', 'purpose', 'title',
        'addr_state', 'dti', 'delinq_2yrs',
        'earliest_cr_line', 'inq_last_6mths', 'open_acc',
        'pub_rec', 'revol_bal', 'revol_util',
        'total_acc', 'initial_list_status', 'last_credit_pull_d',
        'collections_12_mths_ex_med', 'policy_code', 'application_type',
        'chargeoff_within_12_mths', 'pub_rec_bankruptcies', 'tax_liens']]

df.head()

## Choosing Our Label

'loan_status' seems to be an ideal candidate for a columns that we could predict.  Here are the possible values for that column:

In [ ]:
# values and counts for each values in 'loan status'

df.loan_status.value_counts()

Checking the Lending Tree website, we can see an explanation for each possible value:

- <b>Fully Paid</b> - The loan is paid off

- <b>Charged Off</b>: The loan defaulted

- <b>Does not meet the credit policy. Status:Fully Paid:</b> Even though the loan is paid off, the borrower would not meet the criteria for borrowing at the end of the term of his loan

- <b>Does not meet the credit policy. Status:Charged Off:</b> The loan defaulted, and when it did, the borrower did not meet the criteria for borrowing



Since we are only interested in whether the loan was paid off or not, we will have to change the values of the latter two possible values, since they contain additional, unneeded information.

We will also change the value of any paid off loan to a 1, and a charged off loan to a 2.  This way, we have turned our labels into binary values, allowing our model to simply solve a binary classification problem.

In [ ]:
# creating mapping dictionary to replace all values with 1's or 0's
mapping_dict = {'loan_status': {
                   'Does not meet the credit policy. Status:Fully Paid':1,
                   'Does not meet the credit policy. Status:Charged Off':0,
                   'Fully Paid':1,
                   'Charged Off':0}
               }

# mapping onto our datafram
df = df.replace(mapping_dict)

# testing to make sure the results took place
df.loan_status.value_counts()

## Eliminating Single-Value Columns

Any column that doesn't contain more than a single value cannot be used to help us predict anything.
Now we well check each column to make sure there is variation in its data.

In [ ]:
# list to hold single value columns
single_val_cols = []

# checking each column for number of values in its "value_counts" function
for col in df.columns:
    if len(df[col].value_counts()) == 1:
        single_val_cols.append(col)
        
        
print(single_val_cols)

# dropping those columns
df.drop(single_val_cols, axis=1, inplace=True)

## Data Cleaning

Now that we have removed all unnecessary columns and selected a grouping to work with as our features, we will need to thoroughly clean the data.  This will involve tasks such as:

- removing rows/columns with missing data or imputing the missing data


- converting columns to data types we can work with numerically or categorically


- impute new feature columns from processing and combinations of existing columns


- removing any other columns we've chosen that are hard to work with, or won't work as a feature for whatever reason

### Missing Values

First, let's see how many missing values are left in our data set

In [ ]:
# print null values of each columns that have null values

null_col_sums = df.isnull().sum()
null_col_sums[null_col_sums>0].sort_values(ascending=False)

In [ ]:
# print percentage of null values of each columns that have null values

null_col_sums = df.isnull().sum() / len(df)
null_col_sums[null_col_sums>0].sort_values(ascending=False)

Now we will examine our two columns that have the largest number of missing values

<b>pub_rec_bankruptcies<b/> and <b>emp_length<b/>

In [ ]:
# observing the frequency of values in pub_rec_bankruptcies

df.pub_rec_bankruptcies.value_counts(dropna=False)

Although the variation is low in this column, domain knowledge tells us that knowing if someone has declared bankruptcy could be a useful value in assessing their credit history.   We will impute the much more common value of "0" (did not have bankruptcy) into the missing values.

We will also impute "0" into the eight "2.0" values, to allow this column to be a binary.  With extremely little accuracy loss, we can avoid splitting up the column into dummies or drop eight rows of data from the dataset.

In [ ]:
# imputing missing values with "0"

df.pub_rec_bankruptcies.loc[df.pub_rec_bankruptcies.isnull()] = 0


# imputing "0" into values with "2.0"

df.pub_rec_bankruptcies.loc[df.pub_rec_bankruptcies == 2.0] = 0



# Testing to make sure our column is now a binary 

df.pub_rec_bankruptcies.value_counts()


The other column missing data is <b>emp_length</b>.
    
emp_length is a representation of employment length for the borrower.  From domain knowledge, we know the stability of a borrowers job will greatly affect his ability to pay back a loan.  Let's examine the values of this column.

In [ ]:
df.emp_length.value_counts(dropna=False)

## Removing the Rest of the Missing Data

There is much variation and only ~2% missing values.  For this columns, we will drop all rows missing this data.  Since all the rest of the columns barely have missing values at all, we will go ahead and remove all rows with missing data from the set at once.

In [ ]:
# dropping rows missing emp_length

df.dropna(axis=0, inplace=True)


# testing to make sure there are no more missing values

df.isnull().sum()

## Observing the Data From Each Data Type In Remaining Columns

To see if we need to make adjustments to any of the columns, we will break each data type into a group to observe the different values contained within.

### Object Data Types:

In [ ]:
# creating dataframe of object type columns
object_cols = df.select_dtypes(include='object')

#printing first row
object_cols.iloc[0]


These columns may need to be made categorical, but first we will have to look into their value_counts to see how many categories each has.  We may need to invent categories by processing the data if necessary.

- home_ownership - Will need to be made categorical

- verification_status: Indicates whether the borrowing was verified by Lending Tree

- emp_length -  This column will need to be numerical

- term - This column will need to be turned into a categorical column

- addr_state: State of the borrower

- purpose: Provided by the homeowner (reason for the loan)

- title: Loan title from borrower

Columns that seem to require too much work to be made useful:

- earliest_cr_line: We will remove this

- last_credit_pull_d: We will remove this

In [ ]:
# dropping earliest_cr_line and last_credit_pull_d
df.drop(['last_credit_pull_d', 'earliest_cr_line'], axis=1, inplace=True)

In [ ]:
# looping over the remaining cols and printing their value counts
for col in df.select_dtypes(include='object'):
    print(df[col].value_counts())
    print('\n')




- The home_ownership, verification_status, emp_length, and term columns all contain only a few distinct values, so we will convert them to dummy columns to use as categorical features

- The emp_length columns will have to be cleaned first, removing the text before turning it into dummy columns


In [ ]:
# cleaning the emp_length column

# creating dictionary to replace all values

mapping_dict = {'emp_length':{
                "10+ years": 10,
                "9 years": 9,
                "8 years": 8,
                "7 years": 7,
                "6 years": 6,
                "5 years": 5,
                "4 years": 4,
                "3 years": 3,
                "2 years": 2,
                "1 year": 1,
                "< 1 year": 0,
                "n/a": 0}
               }

# replacing values in df
df = df.replace(mapping_dict)


# testing to make sure it took
df.emp_length.value_counts()


The addr_state column contains too many discrete values, since it represents the State the borrower lives in.  This would create ~50 different dummy columns, so we will drop this column.

In [ ]:
# dropping addr_state
df.drop('addr_state', axis=1, inplace=True)

The title and purpose columns contain similar data, but the purpose columns contains fewer discrete values, so we will drop the title column as well.

In [ ]:
# dropping title column
df.drop('title', axis=1, inplace=True)

Now, we can create dummy columns for the remaining home_ownership, verification_status, purpose, and term columns, which will create a single column with 0's or 1's for each category within the column.

Then, once created, we can concatenate the dummy columns back to the main dataframe.

In [ ]:
# creating dummy_cols
dummy_cols = pd.get_dummies(df[['home_ownership', 'verification_status', 'purpose', 'term']])

# concatenating dummy_cols to main df
df = pd.concat([df, dummy_cols], axis=1)

# checking to make sure it worked
df.head()


Now that we have the dummy columns appended to the end of our dataframe, we can delete the original columns.

In [ ]:
# dropping original columns we turned into dummy columns

df.drop(['home_ownership', 'verification_status', 'purpose', 'term'], axis=1, inplace=True)

In [ ]:
# checking to make sure new dummy columns appended and old columns were deleted.

df.head()

In [ ]:
df.info()

## Choosing A Metric For Errors In Our Prediction

We will run into a problem since many more loans are marked as paid than unpaid in our loan_status column.  This is called a "class imbalance" and can throw off machine learning algorithms.  The reason for this is the algorithm can have a very high "success rate" simply by predicting that a loan will be paid off.

We can remedy this situation by analyzing the number of:

 - False Negatives: Loans we predict will default and actually are paid off.  And...


 - False Positives: Loans we predict will be paid off, but instead fall into default.

My minimizing the number of False Negatives and minimizing the number of False Positives, we establish a better metric for how profitable our predictions will be on the production line.

If we have a False Negative, we lose money by not profiting on a loan we should have underwritten.

If we have a False Positive, we lose possibly the entire principal amount of the loan.  Since this number is potentially much larger of a loss, minimizing False positives should be the most important metric of our predictions.

This mean we need to optimize for a high recall rate (true positive rate) and a low fallout rate (false positive rate)

fp = False Positive

fn = False Negative

tp = True Positive

tn = True Negative


False Positive Rate = fp / (fp + tn)

True Positive Rate = tp / (tp + fn)

Generally speaking, as we attempt to reduce the false positive rate (loans we funded but shouldn't have), the rate of true positives will drop as well.  This makes sense, since by being more conservative about making loans in the first place, we are going to miss a few opportunities while avoiding pitfalls. 

## Predictions

Now that we have our data cleaned and process, or goal set, and our error metrics chosen, we can set up our algorithm and run it to see how accurately it can predict which borrowers will default on their loans before we underwrite them.



## Logistic Regression

Since we are making a binary prediction (will borrower pay back their loan or not), Logistic Regression is a good place to start.  It is less likely to over-fit the data as a decision tree, and less computationally intensive as a random forest.

We will be using LogisticRegression from the sklearn.linear_model python library.
We will also be performing kfold cross-validation (with five folds) to ensure that we can optimal separation of testing and training dataframes.  This should help ensure that our results hold up outside of the lab and aren't merely over-fit to the training data.



In [ ]:
# importing modules
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

# creating feature dataset
features = df.drop('loan_status', axis=1)

# making sure features has only 39 columns
print(features.shape[1])

# creating label out of loan status column
label = df.loan_status

Since we will need to calculate our "True Positive Rate" and our "False Positive Rate" after we make our predictions, we will write a function that does it for us.

In [ ]:
# writing function that will take out predictions from our machine learning algoirth
# and will calculate the "True Positive Rate" and our "False Positive Rate" 

def tpr_and_fpr(predictions):
    tp = (df.loan_status == 1) & (predictions == 1)   # true positives
    fn = (df.loan_status == 1) & (predictions == 0)   # false negatives
    fp = (df.loan_status == 0) & (predictions == 1)   # false positives
    tn = (df.loan_status == 0) & (predictions == 0)   # true negatives
    
    tpr = tp.sum() / (tp.sum() + fn.sum())     # true positive rate
    fpr = fp.sum() / (fp.sum() + tn.sum())     # false positive rate
    
    return tpr, fpr

In [ ]:
# performing Logistic Regression and making predictions

# creating Logistic Regression object
lr = LogisticRegression()

# using cross_val_predict to fit the data, perform kfold cross validation and predict
predictions = cross_val_predict(lr, features, label, cv=5, n_jobs=8)


In [ ]:
# calculating our tpr and fpr from our predictions

tpr, fpr = tpr_and_fpr(predictions)

print(tpr, fpr)

## Conclusions

We can see that our Logistic Regression was hardly more useful at making predictions than simply guessing that every person would pay back their loan on time.  This is due to the large class imbalance in the data.  There is too high of a percentage of people that pay back their loans on time in the dataset for machine learning algorithms to make an accurate prediction.

By weighting the rows of data where the label is '0' so that they are taken into account as much as all of the rows where the label is '1', hopefully we can get more accurate predictions.

Fortunately, sklearn has a 'class_weight' parameter that balances each label in proportion to its frequency in the data set.  We can use it by setting passing the argument 'balanced' into the 'class_weight' parameter.

Once we do that, since there are six times as many rows where the loan is paid off, all the rows where the loan *was not* paid off are weighted six times as strongly by the logistic regression.

In [ ]:
# performing weighted Logistic Regression and making predictions

# creating Logistic Regression object
lr = LogisticRegression(class_weight='balanced')

# using cross_val_predict to fit the data, perform kfold cross validation and predict
predictions = cross_val_predict(lr, features, label, cv=5, n_jobs=8)

# calculating our tpr and fpr from our predictions

tpr, fpr = tpr_and_fpr(predictions)

print("TPR: ",tpr)
print("FPR: ", fpr)

## Conclusions

These numbers should make the conservative investor a bit happier.  We are now writing 57% of the profitable loans, while correctly identifying 69% of the loans that would default.  To see if we can improve these numbers, we will see what happens when we manually weight the rows with '0' even higher.

Currently, sklearn has the weight for a '0' label set to ~5.8x the weight for a '1' row.

We will now re-run our logistic regression with the weight increased to 10x for the '0' rows.

In [ ]:
# performing 10x weighted Logistic Regression and making predictions

# creating Logistic Regression object
custom_weight = {0:10, 1:1}
lr = LogisticRegression(class_weight=custom_weight)

# using cross_val_predict to fit the data, perform kfold cross validation and predict
predictions = cross_val_predict(lr, features, label, cv=5, n_jobs=8)

# calculating our tpr and fpr from our predictions

tpr, fpr = tpr_and_fpr(predictions)

print("TPR: ",tpr)
print("FPR: ", fpr)

## Conclusions

By weighting the columns where people defaulted heavier, we were able to eliminate 94% of the customers who would default on their loans, but we also eliminated 85% of our profitable customers, so this is probably too cautious of a model to be realistic to your average investor.

## Random Forest Regressor

We could attempt to optimize our logistic regression model further, but it would be prudent to see how accurately a random forest would compare, before we dive too deeply into logistic regression.  We will now perfom the same prediction steps as before, but replacing logistic regression with random forest.

In [ ]:
# importing random forest
from sklearn.ensemble import RandomForestClassifier

# performing weighted Random Forest and making predictions

# creating random forest object
rf = RandomForestClassifier(class_weight='balanced', random_state=42)

# using cross_val_predict to fit the data, perform kfold cross validation and predict
predictions = cross_val_predict(rf, features, label, cv=5, n_jobs=8)

# calculating our tpr and fpr from our predictions

tpr, fpr = tpr_and_fpr(predictions)

print("TPR: ",tpr)
print("FPR: ", fpr)

Now we will re-run with heavier 8x weights for the '0' rows:

In [ ]:
# performing 8x weighted Random Forest and making predictions

# creating random forest object
custom_weight = {0:8, 1:1}
rf = RandomForestClassifier(class_weight=custom_weight, random_state=42)

# using cross_val_predict to fit the data, perform kfold cross validation and predict
predictions = cross_val_predict(rf, features, label, cv=5, n_jobs=8)

# calculating our tpr and fpr from our predictions

tpr, fpr = tpr_and_fpr(predictions)

print("TPR: ",tpr)
print("FPR: ", fpr)

## Conclusions

Logistic Regression was much more accurate at predicting which customers will pay back their loans.  With more optimization, we could likely improve the accuracy.

If we could link our feature table with a table that calculated the profitability of each loan (referencing the loan_id), we would be able to replace 'profitability' as the error metric instead of tpr and fpr.  